In [1]:
from qiskit import Aer, execute
from qiskit.optimization import QuadraticProgram
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, CobylaOptimizer

S_simulator = Aer.get_backend('statevector_simulator')

# 7. Optimization

---
## Question 7.1

**Which of the following statement is incorrect for this QuadraticProgram?**

```
mod = QuadraticProgram()
mod.binary_var('x')
mod.binary_var('y')
mod.minimize(constant = 3, linear=[1,-2], quadratic={('x', 'y'): 2})
```

**1)** It has two variables

**2)** The objective is to minimise `x - 2 y + 2 x^2 + 2 y^2 + 3`

**3)** It has no constraints

**4)** It is a Quadratic Unconstrained Binary Optimization (QUBO) problem

**5)** None of the above

### Solution to Q7.1

Looking carefully at the arguments for the objective function, we see that the parameters being passed are: 

`constant = 3`, which adds a constant of `3` to the function.

`linear=[1,-2]` adds linear components for the variables `x` and `y` with prefactors `1` and `-2`, respectively.

`quadratic={('x', 'y'): 2}` is a dictionary that adds a quadratic component of `2*x*y`, which in [LP format](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.9.0/ilog.odms.cplex.help/CPLEX/FileFormats/topics/LP_Quadratic.html) corresponds to `[4*x*y]/2`.

This gives the objective function: `x - 2 y + [ 4 x*y ]/2 + 3`, making **option 2** incorrect. We can verify this by running the code for this problem:

In [2]:
mod = QuadraticProgram('my problem')
mod.binary_var('x')
mod.binary_var('y')
mod.minimize(constant = 3, linear=[1,-2], quadratic={('x', 'y'): 2})
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: x - 2 y + [ 4 x*y ]/2 + 3
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1

Binaries
 x y
End



### Answer to Q7.1

**2)** The objective is to minimise `x - 2 y + 2 x^2 + 2 y^2 + 3`

---
## Question 7.2

**What kinds of minimum eigen solvers can be used for the MinimumEigenOptimizer?**

```
problem = QuadraticProgram()
# specify problem here
# specify minimum eigen solver to be used
mes = (*what can be here?*)
optimizer = MinimumEigenOptimizer(mes)
result = optimizer.solve(problem)
```

**1)**  `VQE(...)`

**2)**  `QAOA(...)`

**3)**  `NumpyMinimumEigensolver(...)`

**4)**  All of the above

**5)**  None of the above

### Solution to Q7.2

From the tutorial on the [Minimum Eigen Optimizer](https://qiskit.org/documentation/tutorials/optimization/3_minimum_eigen_optimizer.html): 

"Qiskit provides automatic conversion from a suitable `QuadraticProgram` to an Ising Hamiltonian, which then allows to leverage all the `MinimumEigenSolver` available in Qiskit Aqua, such as - `VQE`, - `QAOA`, or - `NumpyMinimumEigensolver` (classical exact method).

Qiskit wraps the translation to an Ising Hamiltonian (in Qiskit Aqua also called `Operator`), the call to an `MinimumEigensolver` as well as the translation of the results back to `OptimizationResult` in the `MinimumEigenOptimizer`."

This means all of the listed eigen solvers are supported. 

Let's for example run `QAOA` and `NumpyMinimumEigensolver` for the problem defined above for Question 7.1.

In [3]:
# QAOA
qaoa_mes = QAOA(quantum_instance = S_simulator)
qaoa = MinimumEigenOptimizer(qaoa_mes)
qaoa_result = qaoa.solve(mod)
print(qaoa_result)

x=[0.0,1.0], fval=1.0


In [4]:
# MinimumEigenOptimizer
exact_mes = NumPyMinimumEigensolver()
exact = MinimumEigenOptimizer(exact_mes)
exact_result = exact.solve(mod)
print(exact_result)

x=[0.0,1.0], fval=1.0


### Answer to Q7.2

**4)**  All of the above

---
## Question 7.3

**Which optimizers in Qiskit can be used to solve Quadratic Unconstrained Binary Optimization (QUBO) problems?**

**1)** MinimumEigenOptimizer

**2)** GroverOptimizer

**3)** CobylaOptimizer

**4)** Both 1 & 2

**5)** All of the above


### Solution to Q7.3

as discussed in previous problem, the tutorial on the [Minimum Eigen Optimizer](https://qiskit.org/documentation/tutorials/optimization/3_minimum_eigen_optimizer.html) shows different ways this optimizer can be used to solve QUBOs.

In the tutorial on the Grover Optimizer, we can find an example on how to find the [minimum of a QUBO](https://qiskit.org/documentation/tutorials/optimization/4_grover_optimizer.html) using this optimizer.

Lastly, the [`CobylaOptimizer`](https://qiskit.org/documentation/stubs/qiskit.optimization.algorithms.CobylaOptimizer.html?highlight=cobylaoptimizer) utilizes "constrained optimization by linear approximation ([COBYLA](https://qiskit.org/documentation/stubs/qiskit.aqua.components.optimizers.COBYLA.html))" to solve "**constrained problems** where the derivative of the objective function is not known". therefore this optimizer cannot be used to solve QUBO problems.

We can also check if a QUBO problem (like the one for Question 7.1) can by solved by this optimizer, using the `CobylaOptimizer.is_compatible()` function:

In [5]:
cobyla = CobylaOptimizer()
cobyla.is_compatible(mod)

False

### Answer to Q7.3

**4)** Both 1 & 2